In [1]:
import random
import mph
import numpy as np
import matplotlib.pyplot as plt
import os    
import time
from tqdm.notebook import tqdm
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

path = 'D:/Josh/Fall 2021/ML-LWA/1dsim.mph'

scattering = [1,2,5,6,7,51,53,56,57]

In [2]:
def start_client(path):
    global model
    global pymodel
    global client
    client = mph.start(version = '5.3a')
    pymodel = client.load(path)
    model = pymodel.java
    
def save_model():
    model.save(path)
    
def generate_image(random_pattern):
    adjusted = [x - 15 for x in random_pattern]
    out = np.flip(np.array([1 if x in adjusted else 0 for x in np.arange(36)]))
    return out
    
def simulate_random():
    global start_count
#     grids_of_interest = [random.randint(15,50) for i in range(0,random.randint(1,36))]
#     grids_of_interest = [random.randint(15,50) for i in range(0,18)]
    grids_of_interest = random.sample(range(15,51),18)
    this_scattering = scattering + grids_of_interest
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

def simulate(vals):
    global start_count
    grids_of_interest = vals
    this_scattering = scattering + grids_of_interest
#     print(this_scattering)
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

<h1>Please don't close out of this, very long simulation in progress! Using the computer is fine just don't close this window, thanks!</h1>

<h4>Light simulations should work fine but if you need to do anything heavy, click Kernel in the toolbar above and just hit shutdown, just shoot me (Josh) an email if you need to do that.</h4>

In [ ]:
for instance in tqdm(np.arange(1)):
    img_list = []

    save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/1dconstantslots/' + str(int(time.time()))

    start_client(path)
    start_time = time.time()
    for i in tqdm(np.arange(1)):
        goi = simulate_random()
        array = generate_image(goi)
        img_list.append(array)

    print(time.time() - start_time, 'seconds')

    with open(save_str + '.pkl', 'wb') as file:
        pickle.dump(img_list, file)

In [ ]:
model.save('D:/Josh/Fall 2021/ML-LWA/check1dtest.mph')

In [ ]:
plt.imshow(tf.expand_dims(img_list[0],1),cmap='YlGnBu')

In [4]:
with open('1d/peaks_to_slot/constant_power/results/test_data510.pkl', 'rb') as pkl_file:
    slots, true, preds, _ = pickle.load(pkl_file)

In [ ]:
preds.shape

In [5]:
def top_16(slot):

    median = np.sort(slot)[18]
    out = [1 if x >= median else 0 for x in slot]
    return out

In [ ]:
plt.imshow(tf.expand_dims(top_16(preds[12]),1),cmap = 'YlGnBu')

In [ ]:
start_client(path)

In [6]:
img_list = []

save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/validation510'

start_client(path)
start_time = time.time()
for i in tqdm(np.arange(500)):
    
    assump = list(np.where(np.flip(top_16(preds[i])))[0] + 15)

#     naive = list(np.where(np.flip(tf.round(prediction[i])))[0] + 15)
    
    goi = simulate(assump)
    array = generate_image(goi)
    img_list.append(array)
    
#     goi = simulate(naive)
#     array = generate_image(goi)
#     img_list.append(array)

print(time.time() - start_time, 'seconds')

with open(save_str + '.pkl', 'wb') as file:
    pickle.dump(img_list, file)

  0%|          | 0/500 [00:00<?, ?it/s]

18335.179027318954 seconds


In [ ]:
img_list[0]

In [ ]:
plt.imshow(tf.expand_dims(img_list[0],1))

In [ ]:
plt.imshow(tf.expand_dims(top_16(preds[0]),1))

In [ ]:
test = np.array([1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [ ]:
img_list = []

save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/1d/' + str(int(time.time()))

start_client(path)
start_time = time.time()
    
assump = list(np.where(np.flip(test))[0] + 15)
    
goi = simulate(assump)
array = generate_image(goi)
img_list.append(array)



print(time.time() - start_time, 'seconds')

with open(save_str + '.pkl', 'wb') as file:
    pickle.dump(img_list, file)